## Data Cleaning

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .appName("SparkSession") \
     .getOrCreate()

sc = spark.sparkContext 
sc

<SparkContext master=yarn appName=SparkSession>

### Read Full Amazon Review Dataset

In [7]:
amazon_data = spark.read.json("s3://qianyielva/product")

In [8]:
amazon_data.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [9]:
amazon_data.show(10)

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|0000000078|  [1, 1]|    5.0|Conversations wit...|08 11, 2004|A3AF8FFZAZYNE5|                null|          Impactful!|    1092182400|
|0000000116|  [5, 5]|    4.0|Interesting Grish...|04 27, 2002| AH2L9G3DQHHAJ|               chris|  Show me the money!|    1019865600|
|0000000116|  [0, 0]|    1.0|The thumbnail is ...|03 24, 2014|A2IIIDRK3PRRZY|              Helene|Listing is all sc...|    1395619200|
|0000000868|[10, 10]|    4.0|I'll be honest. I...|09 11, 2002|A1TADCM7YWPQ8M|            Joel@AWS|Not a Bad Transla...|    1031702400|
|0000013714|  [0, 0]|    4.0|It had all the so...|10 31

In [22]:
amazon_data.cache()

DataFrame[asin: string, helpful: array<bigint>, overall: double, reviewText: string, reviewTime: string, reviewerID: string, reviewerName: string, summary: string, unixReviewTime: bigint]

In [10]:
amazon_data.count()

82677139

### Read 5-Core Amazon Video Games Dataset

In [11]:
amazon_game = spark.read.json("s3://qianyielva/video")

In [12]:
amazon_game.show(10)

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|0700099867| [8, 12]|    1.0|Installing the ga...| 07 9, 2012|A2HD75EMZR8QLN|                 123|Pay to unlock con...|    1341792000|
|0700099867|  [0, 0]|    4.0|If you like rally...|06 30, 2013|A3UR8NLLY1ZHCX|Alejandro Henao "...|     Good rally game|    1372550400|
|0700099867|  [0, 0]|    1.0|1st shipment rece...|06 28, 2014|A1INA0F5CWW3J4|Amazon Shopper "M...|           Wrong key|    1403913600|
|0700099867| [7, 10]|    3.0|I got this versio...|09 14, 2011|A1DLMTOTHQ4AST|            ampgreen|awesome game, if ...|    1315958400|
|0700099867|  [2, 2]|    4.0|I had Dirt 2 on X...|06 14

In [19]:
amazon_data.createOrReplaceTempView("amazon_fulldata")

In [25]:
amazon_game.createOrReplaceTempView("amazon_gamedata")

### Find distinct Video Game ID from the video game dataset

In [27]:
Vasin = amazon_game.select('asin').rdd.flatMap(lambda x: x).collect()

In [74]:
unique = list(set(Vasin))

In [49]:
from pyspark.sql import Row

In [53]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [78]:
rdd = sc.parallelize(unique)
videogame_id = rdd.map(lambda x: Row(id=x))
videogame_id = sqlContext.createDataFrame(videogame_id)

In [79]:
videogame_id.createOrReplaceTempView("videogame_id")
#videogame_id.count()

In [80]:
videogame_id.show(10)

+----------+
|        id|
+----------+
|B000O5HZTK|
|B00004RC1Y|
|B00003E4GK|
|B002I0J8FI|
|B0009WPZPY|
|B004IK0M4M|
|B00007LVDN|
|B00002STF1|
|B0002A6CQ4|
|B00009YFU5|
+----------+
only showing top 10 rows



### Select all video games reviews from the Amazon full review data

In [82]:
amazon_fullgame = spark.sql("SELECT * FROM amazon_fulldata WHERE asin in (SELECT id FROM videogame_id)")

In [83]:
amazon_fullgame.show(10)

+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|B00002STH3| [0, 0]|    4.0|It works as it is...| 07 5, 2014| AIGJ9P9JNFOA2|           Amanda A.|              Happy!|    1404518400|
|B00002STH3| [5, 5]|    4.0|I got this game a...|07 23, 2000|A1Y63IWKEOXG6F|       Anthony Rogan|High Quality Raci...|     964310400|
|B00002STH3| [0, 0]|    4.0|The Nintendo 64 p...|06 27, 2011|A36UKFV79879MD|          BX Lounger|Good racer for th...|    1309132800|
|B00002STH3| [0, 0]|    4.0|If you are solely...| 05 2, 2014| ARX0Y3X8WZL6V|      Charles Patton|Fun Game, but not...|    1398988800|
|B00002STH3| [2, 7]|    1.0|the graphics is p...|05 11, 2006| 

In [84]:
amazon_fullgame.count()

1075312

In [92]:
amazon_fullgame.write.save("s3://qianyielva/amazon_game_data", format = "json")

### Read Twitch Dataset

In [118]:
schema = StructType([
    StructField("stream_ID", StringType(), True),
    StructField("current_views", StringType(), True),
    StructField("stream_created_time", StringType(), True),
    StructField("game_name", StringType(), True),
    StructField("broadcaster_ID", StringType(), True),
    StructField("broadcaster_name", StringType(), True),
    StructField("delay_setting", StringType(), True),
    StructField("follower_number", StringType(), True),
    StructField("partner_status", StringType(), True),
    StructField("broadcaster_language", StringType(), True),
    StructField("total_views_of_this_broadcaster", StringType(), True),
    StructField("language", StringType(), True),
    StructField("broadcasters_created_time", StringType(), True),
    StructField("playback_bitrate", StringType(), True),
    StructField("source_resolution", StringType(), True),
])

In [119]:
twitch = spark.read.csv('s3://502-project/Twitch_data',sep = "\t", schema=schema)

In [120]:
twitch.show(10)

+-----------+-------------+--------------------+--------------------+--------------+-------------------+-------------+---------------+--------------+--------------------+-------------------------------+--------+-------------------------+----------------+-----------------+
|  stream_ID|current_views| stream_created_time|           game_name|broadcaster_ID|   broadcaster_name|delay_setting|follower_number|partner_status|broadcaster_language|total_views_of_this_broadcaster|language|broadcasters_created_time|playback_bitrate|source_resolution|
+-----------+-------------+--------------------+--------------------+--------------+-------------------+-------------+---------------+--------------+--------------------+-------------------------------+--------+-------------------------+----------------+-----------------+
|13143376000|       206403|2015-02-14T19:00:22Z|   League of Legends|      36029255|          riotgames|           -1|        1375163|          True|                  en|           

### Twitch Data Cleaning

In [121]:
#remove unnecessary columns
twitch = twitch.drop("broadcasters_created_time").drop("stream_ID").drop("delay_setting").drop("playback_bitrate").drop("source_resolution")

In [122]:
twitch = twitch.drop("broadcaster_name").drop("partner_status")

In [123]:
twitch.show(10)

+-------------+--------------------+--------------------+--------------+---------------+--------------------+-------------------------------+--------+
|current_views| stream_created_time|           game_name|broadcaster_ID|follower_number|broadcaster_language|total_views_of_this_broadcaster|language|
+-------------+--------------------+--------------------+--------------+---------------+--------------------+-------------------------------+--------+
|       206403|2015-02-14T19:00:22Z|   League of Legends|      36029255|        1375163|                  en|                      636705939|      en|
|        78525|2015-02-14T11:19:49Z|Counter-Strike: G...|      31239503|         238950|                  en|                       26775824|      pl|
|        47007|2015-02-14T20:00:00Z|           Minecraft|      15554591|         921635|                  en|                       14392009|      en|
|        31156|2015-02-14T17:05:31Z|                H1Z1|      23161357|         690274|      

In [124]:
twitch.write.save("s3://qianyielva/twitch_clean", format = "json")

In [129]:
sc.stop()

In [130]:
spark.stop()